# Исследование надёжности заёмщиков

На основе информации о платёжеспособности клиентов исследовать влияние различных факторов на факт возврата кредита в срок

### Получим данные

In [3]:
import pandas as pd
from pymystem3 import Mystem
m = Mystem()
from collections import Counter

In [4]:
data = pd.read_csv('')

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [7]:
data.describe(include='all')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
count,21525.000000,19351.000000,21525.000000,21525,21525.000000,21525,21525.000000,21525,21525,21525.000000,1.935100e+04,21525
unique,NaN,NaN,NaN,15,NaN,5,NaN,3,8,NaN,NaN,38
top,NaN,NaN,NaN,среднее,NaN,женат / замужем,NaN,F,сотрудник,NaN,NaN,свадьба
freq,NaN,NaN,NaN,13750,NaN,12380,NaN,14236,11119,NaN,NaN,797
mean,0.538908,63046.497661,43.293380,NaN,0.817236,NaN,0.972544,NaN,NaN,0.080883,1.674223e+05,NaN
std,1.381587,140827.311974,12.574584,NaN,0.548138,NaN,1.420324,NaN,NaN,0.272661,1.029716e+05,NaN
min,-1.000000,-18388.949901,0.000000,NaN,0.000000,NaN,0.000000,NaN,NaN,0.000000,2.066726e+04,NaN
25%,0.000000,-2747.423625,33.000000,NaN,1.000000,NaN,0.000000,NaN,NaN,0.000000,1.030532e+05,NaN
50%,0.000000,-1203.369529,42.000000,NaN,1.000000,NaN,0.000000,NaN,NaN,0.000000,1.450179e+05,NaN
75%,1.000000,-291.095954,53.000000,NaN,1.000000,NaN,1.000000,NaN,NaN,0.000000,2.034351e+05,NaN


In [9]:
data.head(3)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья


### Вывод

21525 наблюдений, 12 колонок

Пропущенные значения в days_employed (общий трудовой стаж в днях) и total_income (ежемесячный доход).

Количество пропущенных значений в обеих колонках одинаковое (случайность или взаимосвязь?): 19351

Отрицательные значения в days_employed (общий трудовой стаж в днях). Ошибка ли это в выгрузке? 

Минимальное значение в children = -1. 

Есть дубли (разные регистры) в колонке education: Среднее и среднее

## Предобработка данных

### Обработка пропусков

Предполагаю, что значение в total_income — критически важный фактор, влияющий на факт погашения кредита в срок, поэтому группирую датасет по типу дохода, нахожу среднюю, заменяю пропущенные значения на среднюю по группе дохода. 

Определяю количество пропущенных значений в days_employed: 2174. Перед отпределением среднего значения в days_employed, заменяю отрицателные значения на положительные. Заменяю NaN на среднее. 

In [12]:
unique_income_types = data['income_type'].unique()

for in_type in unique_income_types:
    mean = data[data['income_type'] == in_type]['total_income'].mean()
    data.loc[data['income_type'] == in_type, 'total_income'] = data.loc[data['income_type'] == in_type, 'total_income'].fillna(mean)
    

daysNan = len(data[data['days_employed'].isna()])
    
data['days_employed'] = data['days_employed'].abs()

daysMeanAbc = data['days_employed'].mean()

data['days_employed'] = data['days_employed'].fillna(daysMeanAbc)

daysNan

0

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Благодаря замене пропущенных значений на средние, удалось сохранить все 21525 наблюдений. 

Вероятно, равное количество пропущенных значений в этих двух колонках — не случайность, это неслучайная ошибка, доход и трудовой стаж взаимозависимые показатели.  

### Замена типа данных

In [16]:
data['children'] = data['children'].abs()

data['days_employed'] = data['days_employed'].astype('int64')
data['total_income'] = data['total_income'].astype('int64')

### Вывод

1. Предположив, что отрицательные значения в days_employed (общий трудовой стаж в днях) и min в children = -1 — это ошибка в выгрузке, заменяю отрицательные значения на положительные.
2. Заменяю тип данных days_employed и total_income с float64 на int64 для повышения скорости расчетов и сокращения объема занимаемой памяти.


### Обработка дубликатов

In [17]:
data.duplicated().value_counts()

False    21471
True        54
dtype: int64

In [18]:
data = data.drop_duplicates().reset_index(drop= True)

In [19]:
data['children'].unique()

array([ 1,  0,  3,  2,  4, 20,  5])

In [20]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [21]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [22]:
data['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [24]:
data['income_type'].unique()


array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [25]:
data['education'] = data.education.str.lower()
data['family_status'] = data.family_status.str.lower()
data['income_type'] = data.income_type.str.lower()
data['purpose'] = data.purpose.str.lower()

### Вывод

Подсчитываю сумму дублирующих строк: 54 (иногда показывает 71) и удаляю дубликаты.

Выявляю неявные дубликаты. В колонках с категориальными типами данных проверила и заменила регистры (education, family_status, income_type, purpose), чтобы, например, 'среднее' , 'Среднее', 'СРЕДНЕЕ' были одной категорией. 

## Лемматизация

Создала список из колонки purpose и циклом лемматизировала в отдельный список, ункальные значения.

In [26]:
purpose_text = data['purpose'].tolist()
 
lemmas = []
 
for line in purpose_text:
    lemmas += m.lemmatize(line)
 
unique_lemmas = []
 
for word in lemmas:
    if word not in unique_lemmas:
        unique_lemmas.append(word)
        
print(unique_lemmas)

#print(Counter(unique_lemmas))


['покупка', ' ', 'жилье', '\n', 'приобретение', 'автомобиль', 'дополнительный', 'образование', 'сыграть', 'свадьба', 'операция', 'с', 'на', 'проведение', 'для', 'семья', 'недвижимость', 'коммерческий', 'жилой', 'строительство', 'собственный', 'подержать', 'свой', 'со', 'заниматься', 'сделка', 'подержанный', 'получение', 'высокий', 'профильный', 'сдача', 'ремонт']


Добавляю в таблицу колонку loan_purpose (цели кредита) по которой можно будет категоризоровать заемщиков. 

Суммируя количество записей (21525), убеждаюсь, что все строчки попали в ту или иную категорию.

In [28]:
def lemmatization(row):
    purpose = row['purpose']
    lemmas = m.lemmatize(purpose)
    return lemmas
 
data['lemmas'] = data.apply(lemmatization, axis=1)

 
def categoryLemmasFunction(row): 
    category_lemmas_row = row ['lemmas']

    if 'автомобиль' in category_lemmas_row:
        return 'Автотранспорт'
    
    if 'жилье' in category_lemmas_row:
        return 'Недвижимость'
    
    if 'недвижимость'in category_lemmas_row:
        return 'Недвижимость'
    
    if 'свадьба'in category_lemmas_row:
        return 'Свадьба'
    
    if 'образование' in category_lemmas_row:
        return 'Образование'
 
data['loanPurpose'] = data.apply(categoryLemmasFunction, axis=1)

data.head(5)

data['loanPurpose'].value_counts().sum()

data['loanPurpose'].unique()


array(['Недвижимость', 'Автотранспорт', 'Образование', 'Свадьба'],
      dtype=object)

### Вывод

Лемматизировав цели кредита, получили 4 категории по целям: недвижимость, автотранспорт, образование, свадьба.

### Категоризация данных

На основании данных из пункта "Обработка дубликатов", можно рассмотреть данные в следующих категориях: 

1. Наличие или отсутствие детей — 3 категории: нет детей, 1 ребенок, 2 и более.

2. Наличие или отсутствие образования — 3 категории: высшее + ученая степень, среднее и без высшего образования. 

3. Партнерский статус — все категории, представленные в списке уникальных значений: женат / замужем, гражданский брак, вдовец/вдова, в разводе, не женат/не замужем

4. По способу зарабатывать деньги — все категории, представленные в списке уникальных значений:
сотрудник, пенсионер, компаньон, госслужащий, безработный, предприниматель, студент, в декрете. 

5. По ежемесячному доходу — 5 категорий: до 75000 (86,83-13%), до 112500, до 150 000, 187500. Шаг 75000/2 = 37500.
https://ria.ru/20190520/1553644527.html
"Средняя зарплата до вычета налогов в Москве за 1 квартал 2019 года выросла на 9,5% по сравнению с аналогичным периодом 2018 года и составила 86,83 тысяч рублей, сообщил РИА Новости руководитель департамента экономической политики и развития Москвы Денис Тихонов" 

6. По целям — 4 категории: недвижимость, автотранспорт, образование, свадьба (сделано в предыдущем шаге).

In [30]:
def childrenCount(quantity):
        if quantity == 0:
                return 'Нет детей'
        if quantity <= 1:
                return '1 ребенок'
        if quantity <= 2:    
                return 'Много детей'

data['childrenCount'] = data['children'].apply(childrenCount)


def educationQlassification(row):
        education = row ['education']
        
        if 'высшее' in education:
            return 'Высшее'
        if 'ученая степень' in education:
            return 'Высшее'
        if 'среднее' in education:
            return 'Среднее'
        return 'Без высшего'
 
data['educationQlassification'] = data.apply(educationQlassification, axis=1)

def partnerStatus(row):
        family_status = row ['family_status']
        
        if 'женат / замужем' in family_status:
            return 'Брак'
        if 'гражданский брак' in family_status:
            return 'Гражданский брак'
        if 'вдовец / вдова' in family_status:
            return 'Вдовствующие'
        if 'в разводе' in family_status:
            return 'Разведеные'      
        return 'Одиночка'
    
 
data['partnerStatus'] = data.apply(partnerStatus, axis=1)

def incomeFrom(row):
        income_type = row ['income_type']
        
        if 'госслужащий' in income_type:
            return 'Госслужащий'
        if 'пенсионер' in income_type:
            return 'Пенсионер'
        if 'студент' in income_type:
            return 'Студент'
        if 'в декрете' in income_type:
            return 'Декрет'
        if 'предприниматель' in income_type:
            return 'Предприниматель'
        if 'компаньон' in income_type:
            return 'Компаньон'
        return 'Сотрудник'
 
data['incomeFrom'] = data.apply(incomeFrom, axis=1)


def incomeCount(quantity):
        if quantity < 75000:
                return 'Ниже среднего'
        if quantity <= 112500:
                return 'Средний'
        if quantity <= 150000:
                return 'Выше среднего'
        return 'Значительно выше среднего'

data['incomeCount'] = data['total_income'].apply(incomeCount)

data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas,loanPurpose,childrenCount,educationQlassification,partnerStatus,incomeFrom,incomeCount
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",Недвижимость,1 ребенок,Высшее,Брак,Сотрудник,Значительно выше среднего
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",Автотранспорт,1 ребенок,Среднее,Брак,Сотрудник,Средний
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",Недвижимость,Нет детей,Среднее,Брак,Сотрудник,Выше среднего
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",Образование,None,Среднее,Брак,Сотрудник,Значительно выше среднего
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",Свадьба,Нет детей,Среднее,Гражданский брак,Пенсионер,Значительно выше среднего


### Вывод

На основании уникальный значений, разделила данные на категории, каждую категорию выделила в отдельный столбец. 



### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [35]:
children_pivot = data.pivot_table(index = ['childrenCount'], columns = 'debt', values = 'children', aggfunc='count')

new_columns = ['В срок','Просрочен']
children_pivot.set_axis(new_columns, axis='columns',inplace = True)
 
#создаем новый столбец с долей и выводим таблицу на печать
children_pivot['Доля просроченных платежей']= children_pivot['Просрочен']/(children_pivot['Просрочен']+children_pivot['В срок'])
children_pivot.sort_values(by = 'Доля просроченных платежей', ascending = False).T.style.background_gradient("Reds", axis=1)

childrenCount,Много детей,1 ребенок,Нет детей
В срок,1858.000000,4411.000000,13044.000000
Просрочен,194.000000,445.000000,1063.000000
Доля просроченных платежей,0.094542,0.091639,0.075353


### Вывод

Да, зависимость есть: у людей с детьми доля просроченных платежей больше. Соответственное, чем больше детей, тем выше доля просроченных платежей.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [33]:
family_pivot = data.pivot_table(index = ['partnerStatus'], columns = 'debt', values = 'children', aggfunc='count')

new_columns = ['В срок','Просрочен']
family_pivot.set_axis(new_columns, axis='columns',inplace = True)
 
#создаем новый столбец с долей и выводим таблицу на печать
family_pivot['Доля просроченных платежей']= family_pivot['Просрочен']/(family_pivot['Просрочен']+family_pivot['В срок'])
family_pivot.sort_values(by = 'Доля просроченных платежей', ascending = False).T.style.background_gradient("Reds", axis=1)

partnerStatus,Одиночка,Гражданский брак,Брак,Разведеные,Вдовствующие
В срок,2536.000000,3775.000000,11413.000000,1110.000000,896.000000
Просрочен,274.000000,388.000000,931.000000,85.000000,63.000000
Доля просроченных платежей,0.097509,0.093202,0.075421,0.071130,0.065693


### Вывод

У одиноких людей доля просроченных платежей выше, чем у семейных. У людей в гражданском браке доле просроченных платежей выше, чем у людей в официальном браке и развденных. Вдовствующие наиболее дициплинированные.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [34]:
incomeCount_data=data.pivot_table(index = ['incomeCount'], columns = 'debt', values = 'children', aggfunc='count')
 
new_columns = ['В срок','Просрочен']
incomeCount_data.set_axis(new_columns, axis='columns',inplace = True)
 
incomeCount_data['Доля просроченных платежей']=incomeCount_data['Просрочен']/(incomeCount_data['Просрочен']+incomeCount_data['В срок'])
incomeCount_data.sort_values(by = 'Доля просроченных платежей', ascending = False).T.style.background_gradient("Reds", axis=1)

incomeCount,Выше среднего,Средний,Значительно выше среднего,Ниже среднего
В срок,4169.000000,3783.000000,10049.000000,1729.000000
Просрочен,402.000000,342.000000,861.000000,136.000000
Доля просроченных платежей,0.087946,0.082909,0.078918,0.072922


### Вывод

Да, есть: доля просроченных платежей у людей со средним доходом и доходом выше среднего выше, чем у людей с доходом значительно ниже среднего.

- Как разные цели кредита влияют на его возврат в срок?

In [36]:
loanPurpose_data=data.pivot_table(index = ['loanPurpose'], columns = 'debt', values = 'children', aggfunc='count')

new_columns = ['В срок','Просрочен']
loanPurpose_data.set_axis(new_columns, axis='columns',inplace = True)
 
loanPurpose_data['Доля просроченных платежей']=loanPurpose_data['Просрочен']/(loanPurpose_data['Просрочен']+loanPurpose_data['В срок'])
loanPurpose_data.sort_values(by = 'Доля просроченных платежей', ascending = False).T.style.background_gradient("Reds", axis=1)

loanPurpose,Автотранспорт,Образование,Свадьба,Недвижимость
В срок,3905.000000,3644.000000,2149.000000,10032.000000
Просрочен,403.000000,370.000000,186.000000,782.000000
Доля просроченных платежей,0.093547,0.092177,0.079657,0.072314


### Вывод
Кредиты на образование и приобретение автомобиля невозвращают чаще, чем кредиты на свадьбу или невижимость. 

- Как способ заработка влияет на возврат кредита в срок?

In [37]:
incomeFrom_data=data.pivot_table(index = ['incomeFrom'], columns = 'debt', values = 'family_status_id', aggfunc='count')

new_columns = ['В срок','Просрочен']
incomeFrom_data.set_axis(new_columns, axis='columns',inplace = True)
 
incomeFrom_data['Доля просроченных платежей']=incomeFrom_data['Просрочен']/(incomeFrom_data['Просрочен']+incomeFrom_data['В срок'])

plataNan = len(incomeFrom_data[incomeFrom_data['Просрочен'].isna()])
dolyaNan = len(incomeFrom_data[incomeFrom_data['Доля просроченных платежей'].isna()])

incomeFrom_data['Просрочен'] = incomeFrom_data['Просрочен'].fillna(0)
incomeFrom_data['В срок'] = incomeFrom_data['В срок'].fillna(0)
incomeFrom_data['Доля просроченных платежей'] = incomeFrom_data['Доля просроченных платежей'].fillna(0)

incomeFrom_data.sort_values(by = 'Доля просроченных платежей', ascending = False).T.style.background_gradient("Reds", axis=1)

incomeFrom,Сотрудник,Компаньон,Госслужащий,Пенсионер,Декрет,Предприниматель,Студент
В срок,10031.000000,4704.000000,1371.000000,3621.000000,0.000000,2.000000,1.000000
Просрочен,1062.000000,376.000000,86.000000,216.000000,1.000000,0.000000,0.000000
Доля просроченных платежей,0.095736,0.074016,0.059025,0.056294,0.000000,0.000000,0.000000


### Вывод 
Самые злостные неплатильщики — наемные работники, самые дисциплинированые — госслужащие и пенсионеры. 

- Как образование влияет на возврат кредита в срок?

In [38]:
educationQlassification_data=data.pivot_table(index = ['educationQlassification'], columns = 'debt', values = 'children', aggfunc='count')

new_columns = ['В срок','Просрочен']
educationQlassification_data.set_axis(new_columns, axis='columns',inplace = True)
 
educationQlassification_data['Доля просроченных платежей']=educationQlassification_data['Просрочен']/(educationQlassification_data['Просрочен']+educationQlassification_data['В срок'])
educationQlassification_data.sort_values(by = 'Доля просроченных платежей', ascending = False).T.style.background_gradient("Reds", axis=1)

educationQlassification,Без высшего,Среднее,Высшее
В срок,251.000000,13824.000000,5655.000000
Просрочен,31.000000,1364.000000,346.000000
Доля просроченных платежей,0.109929,0.089808,0.057657


### Вывод
Люди без высшего образования чаще не платят по кредитам, почти вдвое больше, чем с высшим. 

## Общий вывод

Влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок? 

У людей с детьми доля просроченных платежей больше. Чем больше детей, тем выше доля.

У одиноких людей доля просроченных платежей выше, чем у семейных. У людей в гражданском браке доле просроченных платежей выше, чем у людей в официальном браке и развденных. Вдовствующие наиболее дициплинированные.

Доля просроченных платежей у людей со средним доходом и доходом выше среднего выше, чем у людей с доходом значительно ниже среднего.

Кредиты на образование и приобретение автомобиля невозвращают чаще, чем кредиты на свадьбу или невижимость.

Самые злостные неплатильщики — наемные работники, самые дисциплинированые — пенсионеры и госслужащие. 

Люди без высшего образования вдвое чаще не платят по кредитам, чем люди с высшим.